In [1]:
import ollama

In [14]:
prompt = "ประเทศไทยมีกี่จังหวัด"

response = ollama.chat(model="llama3.2", messages=[
    # {"role": "system", "content": "You are rapper. Answer with โย้วๆ"},
    {"role": "user", "content": prompt}
])

response

ChatResponse(model='llama3.2', created_at='2025-03-15T14:46:06.935097Z', done=True, done_reason='stop', total_duration=758955292, load_duration=27919375, prompt_eval_count=32, prompt_eval_duration=502000000, eval_count=9, eval_duration=226000000, message=Message(role='assistant', content='ประเทศไทยมี 76 จังหวัด', images=None, tool_calls=None))

In [15]:
response["message"]["content"]

'ประเทศไทยมี 76 จังหวัด'

#### Create Postgres Database with Docker on Local

Using command

 ```
docker run -d --name pgvector-db -e POSTGRES_USER=myuser -e POSTGRES_PASSWORD=mypassword -e POSTGRES_DB=mydatabase -p 5432:5432 pgvector/pgvector:pg17
 ```

In [27]:
import psycopg2

conn = psycopg2.connect(
    dbname="mydatabase",
    user="myuser",
    password="mypassword",
    host="localhost",
    port="5432"
)

cur = conn.cursor()
# cur.execute("CREATE EXTENSION vector;")
cur.execute("""
    CREATE TABLE IF NOT EXISTS documents (
        id SERIAL PRIMARY KEY,
        content TEXT,
        embedding vector(1024)
    )
""")
conn.commit()
cur.close()
conn.close()

In [28]:
from sentence_transformers import SentenceTransformer

In [31]:
embedder = SentenceTransformer("BAAI/bge-m3")

embedding = embedder.encode("ประเทศไทย มีชื่ออย่างเป็นทางการว่า ราชอาณาจักรไทย เดิมมีชื่อว่า สยาม มีจำนวนประชากรเกือบ 66 ล้านคน มีพื้นที่ 513,115 ตารางกิโลเมตร มีกรุงเทพมหานครเป็นเมืองหลวงและเมืองที่ใหญ่ที่สุด").tolist()

In [32]:
embedding

[0.06241690367460251,
 0.03571312129497528,
 -0.0297442227602005,
 -0.0176023431122303,
 -0.032162174582481384,
 -0.043442800641059875,
 -0.016106434166431427,
 -0.034689560532569885,
 -0.00563363591209054,
 0.03575820475816727,
 -0.027221130207180977,
 0.01435569766908884,
 0.003599082352593541,
 -0.02584454044699669,
 -0.0009735628846101463,
 0.034425266087055206,
 0.0377061702311039,
 0.003905228106305003,
 -0.03867082670331001,
 -0.0018409686163067818,
 0.021480055525898933,
 0.002803544048219919,
 -0.018260551616549492,
 -0.0004179220995865762,
 0.015360420569777489,
 0.015013238415122032,
 -0.003961299080401659,
 0.00277324509806931,
 0.007296585012227297,
 -0.055367983877658844,
 -0.0023213927634060383,
 -0.041934266686439514,
 0.04512559622526169,
 -0.01936163567006588,
 -0.022973015904426575,
 0.020481666550040245,
 -0.009940261021256447,
 -0.037400856614112854,
 -0.017440153285861015,
 -0.021734535694122314,
 0.026317065581679344,
 0.00340336374938488,
 0.030617842450737953,


In [34]:
def add_document(text):
    embedder = SentenceTransformer("BAAI/bge-m3")
    embedding = embedder.encode(text).tolist()
    cur.execute("INSERT INTO documents (content, embedding) VALUES (%s, %s)", (text, embedding))
    conn.commit()

In [35]:
documents = [
    "เปิดหลักสูตรระดับปริญญาตรี 5 สาขาวิชา ได้แก่ สาขาวิชาวิทยาการคอมพิวเตอร์ สาขาวิชานวัตกรรมดิจิทัล สาขาวิชาสารสนเทศการลงทุน สาขาวิชาคอมพิวเตอร์เกมและอีสปอร์ต และสาขาวิชาเทคโนโลยีสื่อสังคม",
    "เปิดหลักสูตรระดับปริญญาโท 3 สาขาวิชา ได้แก่ สาขาวิชาการจัดการความมั่นคงไซเบอร์และเทคโนโลยี สาขาวิชาเทคโนโลยีสื่อสังคม สาขาการจัดการนวัตกรรมดิจิทัล",
    "หลักสูตรระดับปริญญาโท สาขาการจัดการนวัตกรรมดิจิทัล เรียนแบบออนไลน์",
    "หลักสูตรระดับปริญญาโท สาขาวิชาการจัดการความมั่นคงไซเบอร์และเทคโนโลยี และสาขาวิชาเทคโนโลยีสื่อสังคม เรียนเสาร์อาทิตย์",
    "เปิดหลักสูตรระดับปริญญาเอก 2 สาขาวิชา ดังนี้ สาขาวิชาเทคโนโลยีสารสนเทศ และสาขาวิชาเทคโนโลยีสื่อสังคม",
    "คณบดีคือ รศ.ดร. เชฏฐเนติ ศรีสอ้าน",
    "วิทยาลัยเทคโนโลยีสารสนเทศและการสื่อสารแห่งมหาวิทยาลัยรังสิต ก่อตั้งเมื่อปี พ.ศ. 2537 ได้เปิดดำเนินการสอนทั้งในระดับปริญญาตรี ปริญญาโท และปริญญาเอกทางด้านสาขาวิชาที่เกี่ยวข้องทาง ด้านเทคโนโลยีสารสนเทศสำหรับกลุ่มนักศึกษา และคนทำงานที่มีพื้นฐานความรู้ที่หลากหลายและประสบการณ์ที่แตกต่างกัน เพื่อที่จะสามารถตอบสนองต่อความต้องการของสังคม",
    "ปรัชญาคือเหนือกว่าด้วยเทคโนโลยี ก้าวสู่ระดับสากล ปณิธานคือมุ่งมั่นสร้างบัณฑิตไอทีมืออาชีพสู่สังคม และวิสัยทัศน์คือเป็นผู้นำด้านให้บริการการศึกษาทางด้านเทคโนโลยีสารสนเทศ โดยเน้นการวิจัยและสร้างพันธมิตรทางวิชาการเพื่อการพัฒนาอย่างยั่งยืน"
]

conn = psycopg2.connect(
    dbname="mydatabase",
    user="myuser",
    password="mypassword",
    host="localhost",
    port="5432"
)

cur = conn.cursor()

for doc in documents:
    add_document(doc)
    
cur.close()
conn.close()   

In [36]:
len(embedding)

1024

In [51]:
def query_postgresql(query_text, k=3):
    query_embedding = embedder.encode(query_text).tolist()

    conn = psycopg2.connect(
    dbname="mydatabase",
    user="myuser",
    password="mypassword",
    host="localhost",
    port="5432")

    cur = conn.cursor()

    query_embedding_str = "[" + ", ".join(map(str, query_embedding)) + "]"

    sql_query = """
        SELECT content, embedding <=> %s::vector AS similarity_score
        FROM documents
        ORDER BY similarity_score ASC
        LIMIT %s;
    """
    cur.execute(sql_query, (query_embedding_str, k))
    results = cur.fetchall()
    cur.close()
    conn.close()

    return results

In [52]:
results = query_postgresql("คณบดีคือใคร")

In [53]:
results

[('คณบดีคือ รศ.ดร. เชฏฐเนติ ศรีสอ้าน', 0.3971494436263995),
 ('ปรัชญาคือเหนือกว่าด้วยเทคโนโลยี ก้าวสู่ระดับสากล ปณิธานคือมุ่งมั่นสร้างบัณฑิตไอทีมืออาชีพสู่สังคม และวิสัยทัศน์คือเป็นผู้นำด้านให้บริการการศึกษาทางด้านเทคโนโลยีสารสนเทศ โดยเน้นการวิจัยและสร้างพันธมิตรทางวิชาการเพื่อการพัฒนาอย่างยั่งยืน',
  0.6320713758468601),
 ('เปิดหลักสูตรระดับปริญญาตรี 5 สาขาวิชา ได้แก่ สาขาวิชาวิทยาการคอมพิวเตอร์ สาขาวิชานวัตกรรมดิจิทัล สาขาวิชาสารสนเทศการลงทุน สาขาวิชาคอมพิวเตอร์เกมและอีสปอร์ต และสาขาวิชาเทคโนโลยีสื่อสังคม',
  0.7384268769789113)]

In [61]:
import ollama

def generate_response(query_text):
    retrieved_docs = query_postgresql(query_text, 3)

    context = "\n".join([doc[0] for doc in retrieved_docs])

    prompt = f"Answer the question based on the following context:\n{context}\n\nQuestion: {query_text}"
    response = ollama.chat(model="llama3.2", messages=[
        {"role": "system", "content": "You are an assistant"},
        {"role": "user", "content": prompt}
    ])

    return response["message"]["content"]

In [63]:
generate_response("ปริญญาตรีเปิดสอนอะไรบ้าง")

'ปริญญาตรีของโรงเรียนนี้เปิดสอน 5 สาขาวิชา ได้แก่ \n\n1. วิทยาการคอมพิวเตอร์\n2. นวัตกรรมดิจิทัล\n3. สารสนเทศการลงทุน\n4. คอมพิวเตอร์เกมและอีสปอร์ต \n5. เทคโนโลยีสื่อสังคม'